In [34]:
import pandas as pd 

df_prime = pd.read_csv('/data/pj20/txgnn/kg/kg_directed.csv')

/tmp/ipykernel_124327/2610146637.py:3: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_prime = pd.read_csv('/data/pj20/txgnn/kg/kg_directed.csv')


In [35]:
df_prime

x_type              x_id                    relation  \
0        gene/protein            9796.0             protein_protein   
1        gene/protein            7918.0             protein_protein   
2        gene/protein            8233.0             protein_protein   
3        gene/protein            4899.0             protein_protein   
4        gene/protein            5297.0             protein_protein   
...               ...               ...                         ...   
4050244       disease           16982.0  disease_phenotype_positive   
4050245       disease  19314_19023_2726  disease_phenotype_positive   
4050246       disease  19314_19023_2726  disease_phenotype_positive   
4050247       disease            4747.0  disease_phenotype_positive   
4050248       disease            5297.0  disease_phenotype_positive   

                   y_type      y_id    x_idx    y_idx  
0            gene/protein   56992.0  27422.0  19536.0  
1            gene/protein    9240.0  23886.0  26764.0  
2            gene/protein   23548.0  24822.0  10205.0  
3            gene/protein   11253.0  16773.0   5880.0  
4            gene/protein    8601.0  17769.0  25909.0  
...                   ...       ...      ...      ...  
4050244  effect/phenotype   30448.0   4676.0   6378.0  
4050245  effect/phenotype    8069.0   6562.0  13832.0  
4050246  effect/phenotype  100495.0   6562.0    523.0  
4050247  effect/phenotype     202.0  12414.0   4558.0  
4050248  effect/phenotype     795.0  12891.0  13759.0  

[4050249 rows x 7 columns]

In [36]:
df_prime = df_prime[['x_type', 'x_id', 'relation', 'y_type', 'y_id']]

In [37]:
df_drug_src = df_prime[df_prime.x_type == 'drug']
df_drug_dst = df_prime[df_prime.y_type == 'drug']

In [38]:
import numpy as np

np.unique(df_drug_src.y_type.values), np.unique(df_drug_src.relation.values)

(array(['disease', 'drug', 'effect/phenotype', 'gene/protein'],
       dtype=object),
 array(['contraindication', 'drug_drug', 'drug_effect', 'drug_protein',
        'indication', 'off-label use'], dtype=object))

In [39]:
import json

with open('./prime_kg_db_cid.json', 'r') as f:
    db2cid = json.load(f)

with open('./primekg_id_mapping.json' , 'r') as f:
    protein_id2name = json.load(f)['id2name_protein']

with open('./primekg_disease_id_to_dzid.json', 'r') as f:
    disease_id2dzid = json.load(f)

In [40]:
df_drug_src

x_type     x_id      relation            y_type    y_id
321075    drug  DB09130  drug_protein      gene/protein  2157.0
321076    drug  DB09130  drug_protein      gene/protein  2153.0
321077    drug  DB09140  drug_protein      gene/protein  3040.0
321078    drug  DB00180  drug_protein      gene/protein   866.0
321079    drug  DB00240  drug_protein      gene/protein   866.0
...        ...      ...           ...               ...     ...
2076036   drug  DB00105   drug_effect  effect/phenotype  2595.0
2076037   drug  DB00105   drug_effect  effect/phenotype  1618.0
2076038   drug  DB00105   drug_effect  effect/phenotype  1677.0
2076039   drug  DB00105   drug_effect  effect/phenotype  1262.0
2076040   drug  DB00105   drug_effect  effect/phenotype  2791.0

[1469382 rows x 5 columns]

In [41]:
df_drug_src['x_id'] = df_drug_src['x_id'].map(db2cid)
df_drug_src.loc[df_drug_src['y_type'] == 'gene/protein', 'y_id'] = df_drug_src.loc[df_drug_src['y_type'] == 'gene/protein', 'y_id'].map(protein_id2name)

/tmp/ipykernel_124327/2912432475.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drug_src['x_id'] = df_drug_src['x_id'].map(db2cid)


In [42]:
df_drug_src.loc[df_drug_src['y_type'] == 'drug', 'y_id'] = df_drug_src.loc[df_drug_src['y_type'] == 'drug', 'y_id'].map(db2cid)

In [43]:
df_drug_src.loc[df_drug_src['y_type'] == 'disease', 'y_id'] = df_drug_src.loc[df_drug_src['y_type'] == 'disease', 'y_id'].map(disease_id2dzid)

In [44]:
df_drug_src

x_type        x_id      relation            y_type      y_id
321075    drug     23978.0  drug_protein      gene/protein        F8
321076    drug     23978.0  drug_protein      gene/protein        F5
321077    drug       977.0  drug_protein      gene/protein      HBA2
321078    drug     82153.0  drug_protein      gene/protein  SERPINA6
321079    drug   5311000.0  drug_protein      gene/protein  SERPINA6
...        ...         ...           ...               ...       ...
2076036   drug  71306834.0   drug_effect  effect/phenotype    2595.0
2076037   drug  71306834.0   drug_effect  effect/phenotype    1618.0
2076038   drug  71306834.0   drug_effect  effect/phenotype    1677.0
2076039   drug  71306834.0   drug_effect  effect/phenotype    1262.0
2076040   drug  71306834.0   drug_effect  effect/phenotype    2791.0

[1469382 rows x 5 columns]

In [45]:
df = df_drug_src
df[['x_type', 'y_type']] = df[['x_type', 'y_type']].replace('drug', 'molecule')

/tmp/ipykernel_124327/2556175146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['x_type', 'y_type']] = df[['x_type', 'y_type']].replace('drug', 'molecule')


In [46]:
df

x_type        x_id      relation            y_type      y_id
321075   molecule     23978.0  drug_protein      gene/protein        F8
321076   molecule     23978.0  drug_protein      gene/protein        F5
321077   molecule       977.0  drug_protein      gene/protein      HBA2
321078   molecule     82153.0  drug_protein      gene/protein  SERPINA6
321079   molecule   5311000.0  drug_protein      gene/protein  SERPINA6
...           ...         ...           ...               ...       ...
2076036  molecule  71306834.0   drug_effect  effect/phenotype    2595.0
2076037  molecule  71306834.0   drug_effect  effect/phenotype    1618.0
2076038  molecule  71306834.0   drug_effect  effect/phenotype    1677.0
2076039  molecule  71306834.0   drug_effect  effect/phenotype    1262.0
2076040  molecule  71306834.0   drug_effect  effect/phenotype    2791.0

[1469382 rows x 5 columns]

In [47]:
df['score'] = 1.0
df

/tmp/ipykernel_124327/2492878928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = 1.0


x_type        x_id      relation            y_type      y_id  score
321075   molecule     23978.0  drug_protein      gene/protein        F8    1.0
321076   molecule     23978.0  drug_protein      gene/protein        F5    1.0
321077   molecule       977.0  drug_protein      gene/protein      HBA2    1.0
321078   molecule     82153.0  drug_protein      gene/protein  SERPINA6    1.0
321079   molecule   5311000.0  drug_protein      gene/protein  SERPINA6    1.0
...           ...         ...           ...               ...       ...    ...
2076036  molecule  71306834.0   drug_effect  effect/phenotype    2595.0    1.0
2076037  molecule  71306834.0   drug_effect  effect/phenotype    1618.0    1.0
2076038  molecule  71306834.0   drug_effect  effect/phenotype    1677.0    1.0
2076039  molecule  71306834.0   drug_effect  effect/phenotype    1262.0    1.0
2076040  molecule  71306834.0   drug_effect  effect/phenotype    2791.0    1.0

[1469382 rows x 6 columns]

**Descriptor**

In [48]:
desc_lines = []

with open('./compound_descriptor_triple_1.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_2.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_3.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_4.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_5.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_6.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_7.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_8.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_9.txt', 'r') as f:
    desc_lines += f.readlines()
with open('./compound_descriptor_triple_10.txt', 'r') as f:
    desc_lines += f.readlines()

len(desc_lines)

998810

In [49]:
from tqdm import tqdm
import pandas as pd

new_rows = []
for line in tqdm(desc_lines):
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r.lower(), 'y_type': 'value', 'y_id': float(t), 'score': 1.0}
    new_rows.append(new_row)

df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()


100%|██████████| 998810/998810 [00:01<00:00, 855003.32it/s]


In [50]:
df[df.x_id == 977.0]

x_type   x_id              relation        y_type  y_id  score
2        molecule  977.0          drug_protein  gene/protein  HBA2    1.0
558      molecule  977.0          drug_protein  gene/protein   HBB    1.0
9035     molecule  977.0          drug_protein  gene/protein  COX1    1.0
21318    molecule  977.0          drug_protein  gene/protein  NOX1    1.0
1514860  molecule  977.0                  tpsa         value  34.1    1.0
1556489  molecule  977.0  rotatable_bond_count         value   0.0    1.0
2409111  molecule  977.0        tautomer_count         value   1.0    1.0
2420369  molecule  977.0   covalent_unit_count         value   1.0    1.0

**Cooccurence**

In [51]:
import json

mol_mol = {}
with open('./chemical_chemical_1.json', 'r') as f:
    mol_mol.update(json.load(f))

with open('./chemical_chemical_2.json', 'r') as f:
    mol_mol.update(json.load(f))

with open('./chemical_chemical_3.json', 'r') as f:
    mol_mol.update(json.load(f))

with open('./chemical_chemical_4.json', 'r') as f:
    mol_mol.update(json.load(f))

mol_dz = {}
with open('./chemical_disease_1.json', 'r') as f:
    mol_dz.update(json.load(f))

with open('./chemical_disease_2.json', 'r') as f:
    mol_dz.update(json.load(f))

with open('./chemical_disease_3.json', 'r') as f:
    mol_dz.update(json.load(f))

with open('./chemical_disease_4.json', 'r') as f:
    mol_dz.update(json.load(f))

mol_gene = {}
with open('./chemical_gene_1.json', 'r') as f:
    mol_gene.update(json.load(f))

with open('./chemical_gene_2.json', 'r') as f:
    mol_gene.update(json.load(f))

with open('./chemical_gene_3.json', 'r') as f:
    mol_gene.update(json.load(f))

with open('./chemical_gene_4.json', 'r') as f:
    mol_gene.update(json.load(f))

In [52]:
new_rows = []

for key in tqdm(mol_mol.keys()):
    cos = mol_mol[key]
    src = float(key)
    for co in cos:
        dst = float(co[0].replace('CID', ''))
        score = float(co[1])
        new_row = {'x_type': 'molecule', 'x_id': src, 'relation': 'cooccurence', 'y_type': 'molecule', 'y_id': dst, 'score': score}
        new_rows.append(new_row)

df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

100%|██████████| 36010/36010 [00:00<00:00, 225941.11it/s]


In [53]:
new_rows = []

for key in tqdm(mol_dz.keys()):
    cos = mol_dz[key]
    src = float(key)
    for co in cos:
        dst = co[0]
        score = float(co[1])
        new_row = {'x_type': 'molecule', 'x_id': src, 'relation': 'cooccurence', 'y_type': 'disease', 'y_id': dst, 'score': score}
        new_rows.append(new_row)

df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

100%|██████████| 30269/30269 [00:00<00:00, 315113.62it/s]


In [54]:
new_rows = []

for key in tqdm(mol_gene.keys()):
    cos = mol_gene[key]
    src = float(key)
    for co in cos:
        dst = co[0]
        score = float(co[1])
        new_row = {'x_type': 'molecule', 'x_id': src, 'relation': 'cooccurence', 'y_type': 'gene/protein', 'y_id': dst, 'score': score}
        new_rows.append(new_row)

df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

100%|██████████| 29835/29835 [00:00<00:00, 328373.05it/s]


In [55]:
df[df.x_id == 977.0]

x_type   x_id              relation        y_type         y_id  \
2        molecule  977.0          drug_protein  gene/protein         HBA2   
545      molecule  977.0          drug_protein  gene/protein          HBB   
8765     molecule  977.0          drug_protein  gene/protein         COX1   
20521    molecule  977.0          drug_protein  gene/protein         NOX1   
1318485  molecule  977.0                  tpsa         value         34.1   
1360114  molecule  977.0  rotatable_bond_count         value          0.0   
1393877  molecule  977.0        tautomer_count         value          1.0   
1397415  molecule  977.0   covalent_unit_count         value          1.0   
1412619  molecule  977.0           cooccurence      molecule        962.0   
1412620  molecule  977.0           cooccurence      molecule        947.0   
1412621  molecule  977.0           cooccurence      molecule    5462310.0   
1412622  molecule  977.0           cooccurence      molecule        280.0   
1412623  molecule  977.0           cooccurence      molecule       5793.0   
1587203  molecule  977.0           cooccurence       disease     DZID6914   
1587204  molecule  977.0           cooccurence       disease     DZID8198   
1587205  molecule  977.0           cooccurence       disease     DZID6942   
1587206  molecule  977.0           cooccurence       disease    DZID10277   
1587207  molecule  977.0           cooccurence       disease     DZID8607   
1719805  molecule  977.0           cooccurence  gene/protein  EC_1.15.1.1   
1719806  molecule  977.0           cooccurence  gene/protein          CAT   
1719807  molecule  977.0           cooccurence  gene/protein        HIF1A   
1719808  molecule  977.0           cooccurence  gene/protein  EC_1.10.3.9   
1719809  molecule  977.0           cooccurence  gene/protein        VEGFA   

            score  
2        1.000000  
545      1.000000  
8765     1.000000  
20521    1.000000  
1318485  1.000000  
1360114  1.000000  
1393877  1.000000  
1397415  1.000000  
1412619  1.000000  
1412620  0.994681  
1412621  0.862397  
1412622  0.699679  
1412623  0.676426  
1587203  1.000000  
1587204  0.393408  
1587205  0.308140  
1587206  0.223410  
1587207  0.200368  
1719805  1.000000  
1719806  0.814292  
1719807  0.456201  
1719808  0.454179  
1719809  0.453207

**Neighbor**

In [56]:
with open('./chemical_nbr2d.json', 'r') as f:
    mol_nbr2d = json.load(f)

with open('./chemical_nbr3d.json', 'r') as f:
    mol_nbr3d = json.load(f)

In [57]:
new_rows = []

for key in tqdm(mol_nbr2d.keys()):
    if mol_nbr2d[key] == None:
        continue
    if type(mol_nbr2d[key]) == list:
        nbr = float(mol_nbr2d[key][0])
    else:
        nbr = float(mol_nbr2d[key])
    src = float(key)
    new_row = {'x_type': 'molecule', 'x_id': src, 'relation': 'neighbor_2d', 'y_type': 'molecule', 'y_id': nbr, 'score': 1.0}
    new_rows.append(new_row)

df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

100%|██████████| 56679/56679 [00:00<00:00, 898236.08it/s]


In [58]:
new_rows = []

for key in tqdm(mol_nbr3d.keys()):
    if mol_nbr3d[key] == None:
        continue
    if type(mol_nbr3d[key]) == list:
        nbr = float(mol_nbr3d[key][0])
    else:
        nbr = float(mol_nbr3d[key])
    src = float(key)
    new_row = {'x_type': 'molecule', 'x_id': src, 'relation': 'neighbor_3d', 'y_type': 'molecule', 'y_id': nbr, 'score': 1.0}
    new_rows.append(new_row)

df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

100%|██████████| 51665/51665 [00:00<00:00, 910781.99it/s]


In [59]:
df[df.x_id == 23978.0]

x_type     x_id      relation        y_type        y_id     score
0        molecule  23978.0  drug_protein  gene/protein          F8  1.000000
1        molecule  23978.0  drug_protein  gene/protein          F5  1.000000
426      molecule  23978.0  drug_protein  gene/protein         ALB  1.000000
600      molecule  23978.0  drug_protein  gene/protein         A2M  1.000000
680      molecule  23978.0  drug_protein  gene/protein          CP  1.000000
...           ...      ...           ...           ...         ...       ...
1719802  molecule  23978.0   cooccurence  gene/protein       ATP7B  0.460951
1719803  molecule  23978.0   cooccurence  gene/protein  EC_1.9.3.1  0.352543
1719804  molecule  23978.0   cooccurence  gene/protein         CAT  0.332801
1849158  molecule  23978.0   neighbor_2d      molecule        78.0  1.000000
1902652  molecule  23978.0   neighbor_3d      molecule        78.0  1.000000

[207 rows x 6 columns]

**Same_connectivity**

In [60]:
with open('./compound_connectivity.txt', 'r') as f:
    lines = f.readlines()

new_rows = []
for line in lines:
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r, 'y_type': 'molecule', 'y_id': float(t), 'score': 1.0}
    new_rows.append(new_row)


df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

**Component**

In [61]:
with open('./compound_component.txt', 'r') as f:
    lines = f.readlines()

new_rows = []
for line in lines:
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r, 'y_type': 'molecule', 'y_id': float(t), 'score': 1.0}
    new_rows.append(new_row)


df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

**Isotopologue**

In [62]:
with open('./compound_isotopologue.txt', 'r') as f:
    lines = f.readlines()

new_rows = []
for line in lines:
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r, 'y_type': 'molecule', 'y_id': float(t), 'score': 1.0}
    new_rows.append(new_row)


df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

**Parent**

In [63]:
with open('./compound_parent.txt', 'r') as f:
    lines = f.readlines()

new_rows = []
for line in lines:
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r, 'y_type': 'molecule', 'y_id': float(t), 'score': 1.0}
    new_rows.append(new_row)


df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

**Stereoisomer**

In [64]:
with open('./compound_stereoisomer.txt', 'r') as f:
    lines = f.readlines()

new_rows = []
for line in lines:
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r, 'y_type': 'molecule', 'y_id': float(t), 'score': 1.0}
    new_rows.append(new_row)


df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

**Compound2drug**

In [65]:
with open('./compound_todrug.txt', 'r') as f:
    lines = f.readlines()

new_rows = []
for line in lines:
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r, 'y_type': 'drug', 'y_id': t, 'score': 1.0}
    new_rows.append(new_row)


df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

**Compound2type**

In [66]:
with open('./compound_totype.txt', 'r') as f:
    lines = f.readlines()

new_rows = []
for line in lines:
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r, 'y_type': 'drug', 'y_id': t, 'score': 1.0}
    new_rows.append(new_row)


df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

**Pathway**

In [67]:
with open('./compound_pathway.txt', 'r') as f:
    lines = f.readlines()

new_rows = []
for line in lines:
    h, r, t = line[:-1].split('\t')
    new_row = {'x_type': 'molecule', 'x_id': float(h), 'relation': r, 'y_type': 'pathway', 'y_id': t, 'score': 1.0}
    new_rows.append(new_row)


df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
df = df.drop_duplicates()

In [68]:
df[df.x_id == 977.0]

x_type   x_id      relation        y_type         y_id  score
2        molecule  977.0  drug_protein  gene/protein         HBA2    1.0
545      molecule  977.0  drug_protein  gene/protein          HBB    1.0
8765     molecule  977.0  drug_protein  gene/protein         COX1    1.0
20521    molecule  977.0  drug_protein  gene/protein         NOX1    1.0
1318485  molecule  977.0          tpsa         value         34.1    1.0
...           ...    ...           ...           ...          ...    ...
2525546  molecule  977.0    in_pathway       pathway    PWID24198    1.0
2525812  molecule  977.0    in_pathway       pathway  PWID1184698    1.0
2526069  molecule  977.0    in_pathway       pathway  PWID1184675    1.0
2526370  molecule  977.0    in_pathway       pathway  PWID1282200    1.0
2526659  molecule  977.0    in_pathway       pathway  PWID1184528    1.0

[1248 rows x 6 columns]

In [69]:
df.to_csv("KG.csv", index=False)

In [70]:
import pandas as pd

df = pd.read_csv("KG.csv")

# Select rows where relation is 'cooccurrence'
mask = (df['relation'] == 'cooccurence')

# Update the value of the 'relation' column for these rows
df.loc[mask, 'relation'] = 'cooccurence_' + df.loc[mask, 'x_type'] + '_' + df.loc[mask, 'y_type']


/tmp/ipykernel_124327/2894252720.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("KG.csv")


In [71]:
df.relation.unique()

array(['drug_protein', 'contraindication', 'indication', 'off-label use',
       'drug_drug', 'drug_effect', 'defined_bond_stereo_count', 'tpsa',
       'rotatable_bond_count', 'xlogp3-aa', 'structure_complexity',
       'covalent_unit_count', 'defined_atom_stereo_count',
       'molecular_weight', 'hydrogen_bond_donor_count',
       'undefined_bond_stereo_count', 'isotope_atom_count', 'exact_mass',
       'mono_isotopic_weight', 'total_formal_charge',
       'hydrogen_bond_acceptor_count', 'non-hydrogen_atom_count',
       'tautomer_count', 'undefined_atom_stereo_count', 'xlogp3',
       'cooccurence_molecule_molecule', 'cooccurence_molecule_disease',
       'cooccurence_molecule_gene/protein', 'neighbor_2d', 'neighbor_3d',
       'has_same_connectivity', 'has_component', 'has_isotopologue',
       'has_parent', 'has_stereoisomer', 'to_drug', 'closematch', 'type',
       'in_pathway'], dtype=object)

In [72]:
df.to_csv("KG.csv", index=False)